In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))


import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

Running on Apache Spark version 2.4.3
SparkUI available at http://p1427.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.30-2ae07d872f43
LOGGING: writing to /net/archive/groups/plggneuromol/GTS-analysis/analysis/hail-20201116-1454-0.2.30-2ae07d872f43.log


In [79]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [80]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat

Loading BokehJS ...

In [81]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

## Load SKAT functions

In [5]:
def remove_sex_chrom(mtx):
    mtx = mtx.filter_rows(mtx.locus.contig != "chrY")
    return(mtx)


def run_pca(mtx, mtx_subset):
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(mtx_subset.GT)
    mtx = mtx.annotate_cols(scores = pcs[mtx.s].scores)
    p = hl.plot.scatter(mtx.scores[0],
                    mtx.scores[1],
                    label=mtx.phenotypes.family,
                    title='PCA', xlabel='PC1', ylabel='PC2')

    p2 = hl.plot.scatter(mtx.scores[2],
                        mtx.scores[3],
                        label=mtx.phenotypes.family,
                        title='PCA', xlabel='PC3', ylabel='PC4')

    p3 = hl.plot.scatter(mtx.scores[4],
                        mtx.scores[5],
                        label=mtx.phenotypes.family,
                        title='PCA', xlabel='PC5', ylabel='PC6')
    
    p4 = hl.plot.scatter(mtx.scores[6],
                        mtx.scores[7],
                        label=mtx.phenotypes.family,
                        title='PCA', xlabel='PC7', ylabel='PC8')
        
    #show(p)
    #show(p2)
    #show(p3)
    #show(p4)
    
    return(mtx)

def remove_related(mtx, mtx_subset):
    pc_rel = hl.pc_relate(mtx_subset.GT, 0.001, k=2, statistics='kin')
    pairs = pc_rel.filter(pc_rel['kin'] > 0.125)
    related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, keep=False)
    
    related_samples_to_remove = related_samples_to_remove.annotate(s = related_samples_to_remove.node.s)
    related_samples_to_remove = related_samples_to_remove.key_by('s')
    
    mtx = mtx.key_cols_by()
    mtx = mtx.filter_cols(hl.is_defined(related_samples_to_remove[mtx.s]), keep=False)
    mtx = mtx.key_cols_by(mtx.s)
    
    mtx_subset = mtx_subset.filter_cols(hl.is_defined(related_samples_to_remove[mtx_subset.s]), keep=False)
    
    return(mtx, mtx_subset)

def run_skat_log(mtx, gene_list, pcs): #does not have a family covariate
    
    mtx = mtx.filter_rows(hl.any(lambda x: hl.literal(gene_list).contains(x), mtx.nearest_genes_20kb))
    mtx = mtx.filter_rows(hl.agg.any(mtx.GT.is_non_ref()))
    mtx = mtx.explode_rows(mtx.nearest_genes_20kb)
    mtx = mtx.filter_rows(hl.literal(gene_list).contains(mtx.nearest_genes_20kb))
    
    
    scores = [mtx.scores[x] for x in list(range(pcs))]
                          
    
    skat_table = hl.skat(
                         key_expr=mtx.nearest_genes_20kb,
                         weight_expr=mtx.cadd,
                         y=mtx.category,
                         x=mtx.GT.n_alt_alleles(),
                         covariates=[1] + scores,
                         max_size = 2500,
                         logistic = True)
    
    genes_result = skat_table.filter(skat_table.p_value < 0.05/len(gene_list)).id.collect() 

    skat_table.filter(skat_table.p_value < 0.002).show(20)

    skat_table = skat_table.annotate(label = hl.literal(genes).contains(skat_table.id))

    qq_plot = hl.plot.qq(skat_table.p_value,
                                         label = skat_table.label,
                                         n_divisions = len(gene_list))
    show(qq_plot)
    
    return(skat_table, genes_result, qq_plot)


def full_skat_log(mtx, mtx_subset, gene_list, pcs):
    
    mtx = remove_sex_chrom(mtx)
    mtx_subset = remove_sex_chrom(mtx_subset)
  
    mtx = run_pca(mtx, mtx_subset) #this matrix will be returned, so I can do SKAT with other list and parameters
    skat_table, genes_result, qq_plot = run_skat_log(mtx, gene_list, pcs)
    
    return(mtx, skat_table, genes_result, qq_plot)

In [6]:
mt_for_skat = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-skat.mt')
mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-test.mt')
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-subset.mt')
mt_subset_2 = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-subset-2.mt')
mt_subset3 = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-subset-3.mt')

In [7]:
def test_model(top_genes):
    
    variants_controls = np.zeros((len(top_genes)))
    variants_gts = np.zeros((len(top_genes)))

    variants_controls_test = np.zeros((len(top_genes)))
    variants_gts_test = np.zeros((len(top_genes)))

    model_asignment = np.zeros((len(top_genes), 78))
    test_asignment = np.zeros((len(top_genes), 144))  

    for rows, n in enumerate(top_genes):

        mt_skat_log = mt_for_skat.filter_rows(mt_for_skat.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
        mt_test_skat_log = mt_test.filter_rows(mt_test.nearest_genes_20kb.contains(n)) # to ma rodziny

        mt_skat_log = mt_skat_log.filter_rows(mt_skat_log.cadd > 10)
        mt_skat_log = mt_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
        non_refs = mt_skat_log.non_refs.collect()

        mt_test_skat_log = mt_test_skat_log.filter_rows(mt_test_skat_log.cadd > 10)
        mt_test_skat_log = mt_test_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log.GT.is_non_ref())) #count variants per sample, prepare also test matrix
        non_refs_test = mt_test_skat_log.non_refs.collect()

        variants_gts[rows] = np.mean(np.array(non_refs)[categories])
        variants_controls[rows] = np.mean(np.array(non_refs)[np.invert(categories)])

        variants_gts_test[rows] = np.mean(np.array(non_refs_test)[categories_test])
        variants_controls_test[rows] = np.mean(np.array(non_refs_test)[np.invert(categories_test)])

        results = (non_refs - variants_controls[rows]) 
        results_test = (non_refs_test - variants_controls[rows]) 

        model_asignment[rows] = (results)
        test_asignment[rows] = (results_test)

    model_asignment = np.sum(model_asignment, axis = 0)
    test_asignment = np.sum(test_asignment, axis = 0)

    false_pos = []
    true_pos = []

    for x in np.linspace(-100,100,1000):
        false_pos.append(np.sum((test_asignment > x)[np.invert(categories_test)])/53)
        true_pos.append(np.sum((test_asignment > x)[categories_test])/91)
        
    y = np.linspace(0,1,10)
    x = np.linspace(0,1,10)

    from bokeh.plotting import figure, output_notebook, show

    output_notebook

    p = figure(plot_width=800, plot_height=800)


    p.line(x, y, line_width=4, line_color='lightgrey')
    p.line(false_pos, true_pos, line_width=4, alpha=0.5)

    p.xaxis.axis_label = 'false positives'
    p.yaxis.axis_label = 'true positives'

    p.yaxis.axis_label_text_font_size = "25px"
    p.xaxis.axis_label_text_font_size = "25px"


    # show the results
    show(p)
    
    auc = -np.trapz(true_pos, false_pos)
        
    return(false_pos, true_pos, p, auc)

In [8]:
def full_model(gene_list):
    
    mt_for_model, skat_table, genes_result, qq_plot = full_skat_log(mt_for_skat, mt_subset, gene_list, 7)
    skat_table, genes_result, qq_plot = run_skat_log(mt_for_model, gene_list, 7)
    top_genes = skat_table.order_by('p_value').id.take(4)
    false_pos, true_pos, p, auc = test_model(top_genes)
    return(skat_table, qq_plot, top_genes, false_pos, true_pos, p, auc)

### SKAT overrepresentation analysis:

In [9]:
genes = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5'] # the other gene next to chadl - 'L3MBTL2' was deleted not to confuse the analysis


GTS_genes = ['PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'OPRK1', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2'] # a list manually curated from literature

allgenes = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/gts_gene_lists/human-genes-with-GO-and-symbols') 
allgenes = allgenes.select('UniProtKB Gene Name symbol')

allgenes = allgenes.filter(allgenes['UniProtKB Gene Name symbol'] != "")
allgenes = allgenes['UniProtKB Gene Name symbol'].collect()

genes_scores = list(set(genes + GTS_genes))

2020-11-16 14:54:46 Hail: INFO: Reading table with no type imputation
  Loading column 'Gene stable ID' as type 'str' (type not specified)
  Loading column 'UniProtKB Gene Name symbol' as type 'str' (type not specified)



In [82]:
len(genes_scores)

61

In [8]:
#mt_for_skat, mt_subset = remove_related(mt_for_skat, mt_subset) #this needs to be repeated

2020-11-04 13:32:06 Hail: INFO: hwe_normalized_pca: running PCA using 10053 variants.
2020-11-04 13:32:10 Hail: INFO: pca: running PCA with 2 components...


FatalError: IllegalStateException: ARPACK returns non-zero info = 1 Maximum number of iterations taken. (Refer ARPACK user guide for details)

Java stack trace:
java.lang.IllegalStateException: ARPACK returns non-zero info = 1 Maximum number of iterations taken. (Refer ARPACK user guide for details)
	at org.apache.spark.mllib.linalg.EigenValueDecomposition$.symmetricEigs(EigenValueDecomposition.scala:114)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computeSVD(RowMatrix.scala:269)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computeSVD(RowMatrix.scala:208)
	at org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix.computeSVD(IndexedRowMatrix.scala:227)
	at is.hail.methods.PCA.execute(PCA.scala:43)
	at is.hail.expr.ir.functions.WrappedMatrixToTableFunction.execute(RelationalFunctions.scala:51)
	at is.hail.expr.ir.TableToTableApply.execute(TableIR.scala:1777)
	at is.hail.expr.ir.Interpret$.apply(Interpret.scala:24)
	at is.hail.expr.ir.TableIR$$anonfun$persist$1.

In [11]:
mt_for_model, skat_table_log, genes_result_log, qq_plot_log = full_skat_log(mt_for_skat, mt_subset, genes_scores, 7)

2020-11-16 11:49:21 Hail: INFO: hwe_normalized_pca: running PCA using 10045 variants.
2020-11-16 11:49:26 Hail: INFO: pca: running PCA with 10 components...
2020-11-16 11:50:04 Hail: INFO: Coerced sorted dataset
2020-11-16 11:50:04 Hail: INFO: Coerced dataset with out-of-order partitions.
2020-11-16 11:50:18 Hail: INFO: Coerced sorted dataset
2020-11-16 11:50:18 Hail: INFO: Coerced dataset with out-of-order partitions.


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""HDC""",118,5.46e+02,1.79e-03,0


2020-11-16 11:50:33 Hail: INFO: Coerced sorted dataset
2020-11-16 11:50:33 Hail: INFO: Coerced dataset with out-of-order partitions.
2020-11-16 11:50:33 Hail: INFO: Coerced sorted dataset
2020-11-16 11:50:33 Hail: INFO: Coerced dataset with out-of-order partitions.


In [12]:
plot = hl.plot.qq(skat_table_log.p_value)

2020-11-16 11:50:46 Hail: INFO: Coerced sorted dataset
2020-11-16 11:50:46 Hail: INFO: Coerced dataset with out-of-order partitions.
2020-11-16 11:50:46 Hail: INFO: Coerced sorted dataset
2020-11-16 11:50:46 Hail: INFO: Coerced dataset with out-of-order partitions.


In [13]:
show(plot)

In [14]:
skat_table_log.order_by('p_value').show(20)

2020-11-16 11:51:00 Hail: INFO: Coerced sorted dataset
2020-11-16 11:51:00 Hail: INFO: Coerced dataset with out-of-order partitions.


id,size,q_stat,p_value,fault,label
str,int32,float64,float64,int32,bool
"""HDC""",118,5.46e+02,1.79e-03,0,false
"""CHADL""",98,8.35e+02,8.29e-03,0,true
"""MAOA""",95,6.96e+02,1.35e-02,0,false
"""NAA11""",173,6.89e+02,2.41e-02,0,true
"""DRD3""",194,5.51e+02,3.48e-02,0,false
"""DRD2""",224,7.19e+02,6.59e-02,0,false
"""SLITRK2""",81,2.97e+02,9.62e-02,0,false
"""DBH""",265,4.81e+02,1.07e-01,0,false
"""SLITRK4""",82,5.61e+02,1.43e-01,0,false


#### Prepare matrix table

In [158]:
#skat_table_log.write('/net/archive/groups/plggneuromol/GTS-analysis/data/skat.ht') # 7 PCs, zero samples deleted from the analysis

2020-11-13 19:45:49 Hail: INFO: Coerced sorted dataset
2020-11-13 19:45:49 Hail: INFO: Coerced dataset with out-of-order partitions.
2020-11-13 19:45:50 Hail: INFO: wrote table with 53 rows in 53 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/skat-top.ht


In [ ]:
#mt_for_model.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-model.mt')

### Collect the number of variants in control individuals for the classifier

A few iterations over various numbers of genes and CADD score cutt-offs were run

In [15]:
skat = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/data/skat.ht')

In [16]:
skat.order_by('p_value').show(20)

id,size,q_stat,p_value,fault,label
str,int32,float64,float64,int32,bool
"""HDC""",118,5.46e+02,1.79e-03,0,false
"""CHADL""",98,8.35e+02,8.29e-03,0,true
"""MAOA""",95,6.96e+02,1.35e-02,0,false
"""NAA11""",173,6.89e+02,2.41e-02,0,true
"""DRD3""",194,5.51e+02,3.48e-02,0,false
"""DRD2""",224,7.19e+02,6.59e-02,0,false
"""SLITRK2""",81,2.97e+02,9.62e-02,0,false
"""DBH""",265,4.81e+02,1.07e-01,0,false
"""SLITRK4""",82,5.61e+02,1.43e-01,0,false


### reimport the matrixtable again and prepare the test dataset


In [116]:
mt_for_skat = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-skat.mt')

In [18]:
skat.order_by('p_value').id.take(4)

['HDC', 'CHADL', 'MAOA', 'NAA11']

In [119]:
top_genes = ['HDC', 'CHADL', 'MAOA', 'NAA11'] #7 PCs

In [174]:
categories = mt_for_skat.category.collect()
categories_test = mt_test.category.collect()

### test the test dataset - 4 genes, CADD > 10

#### how to calculate confusion matrix values

sum(asignment[np.invert(categories)]) #number of false positives

sum(np.invert(asignment)[categories]) # number of false negatives
        
sum(np.invert(asignment)[np.invert(categories)]) #number of true negatives

sum(asignment[categories]) #number of true positives 

In [21]:
variants_controls = np.zeros((len(top_genes)))
variants_gts = np.zeros((len(top_genes)))

variants_controls_test = np.zeros((len(top_genes)))
variants_gts_test = np.zeros((len(top_genes)))

model_asignment = np.zeros((len(top_genes), 78))
test_asignment = np.zeros((len(top_genes), 144))  

for rows, n in enumerate(top_genes):
        
    mt_skat_log = mt_for_skat.filter_rows(mt_for_skat.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
    mt_test_skat_log = mt_test.filter_rows(mt_test.nearest_genes_20kb.contains(n)) # to ma rodziny

    mt_skat_log = mt_skat_log.filter_rows(mt_skat_log.cadd > 10)
    mt_skat_log = mt_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
    non_refs = mt_skat_log.non_refs.collect()

    mt_test_skat_log = mt_test_skat_log.filter_rows(mt_test_skat_log.cadd > 10)
    mt_test_skat_log = mt_test_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log.GT.is_non_ref())) #count variants per sample, prepare also test matrix
    non_refs_test = mt_test_skat_log.non_refs.collect()

    variants_gts[rows] = np.mean(np.array(non_refs)[categories])
    variants_controls[rows] = np.mean(np.array(non_refs)[np.invert(categories)])

    variants_gts_test[rows] = np.mean(np.array(non_refs_test)[categories_test])
    variants_controls_test[rows] = np.mean(np.array(non_refs_test)[np.invert(categories_test)])

    results = (non_refs - variants_controls[rows]) 
    results_test = (non_refs_test - variants_controls[rows]) 

    model_asignment[rows] = (results)
    test_asignment[rows] = (results_test)

model_asignment = np.sum(model_asignment, axis = 0)
test_asignment = np.sum(test_asignment, axis = 0)

false_pos = []
true_pos = []

for x in np.linspace(-25,25,600):
    false_pos.append(np.sum((test_asignment > x)[np.invert(categories_test)])/53)
    true_pos.append(np.sum((test_asignment > x)[categories_test])/91)

In [22]:
np.save('variants_gts', variants_gts)
np.save('variants_controls', variants_controls)

np.save('variants_gts_test', variants_gts_test)
np.save('variants_controls_test', variants_controls_test)

np.save('model_asignment', model_asignment)
np.save('test_asignment', test_asignment)

In [83]:
variants_gts = np.load('variants_gts.npy')
variants_controls = np.load('variants_controls.npy')

variants_gts_test = np.load('variants_gts_test.npy')
variants_controls_test = np.load('variants_controls_test.npy')

model_asignment = np.load('model_asignment.npy')
test_asignment = np.load('test_asignment.npy')

In [85]:
variants_gts > variants_controls

array([False,  True,  True, False])

In [87]:
variants_gts_test

array([2.3956044 , 3.45054945, 0.40659341, 6.35164835])

In [88]:
variants_controls_test

array([2.20754717, 2.79245283, 0.39622642, 6.28301887])

In [25]:
np.save('false_pos', false_pos)
np.save('true_pos', true_pos)

In [95]:
false_pos = np.load('false_pos.npy')
true_pos = np.load('true_pos.npy')

In [112]:
np.where((true_pos - false_pos) > 0.170)

(array([271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282]),)

In [198]:
true_pos[282]

0.7362637362637363

In [114]:
false_pos[271]

0.5660377358490566

In [115]:
np.linspace(-25,25,600)[271]

-2.378964941569283

In [118]:
len(test_asignment)

144

### ROC

In [91]:
y = np.linspace(0,1,10)
x = np.linspace(0,1,10)

from bokeh.plotting import figure, output_notebook, show

output_notebook

p5 = figure(plot_width=800, plot_height=800)


p5.line(x, y, line_width=4, line_color='lightgrey')
p5.line(false_pos, true_pos, line_width=4, alpha=0.5)

p5.xaxis.axis_label = 'false positives'
p5.yaxis.axis_label = 'true positives'

p5.yaxis.axis_label_text_font_size = "25px"
p5.xaxis.axis_label_text_font_size = "25px"


# show the results
show(p5)

In [28]:
auc = -np.trapz(true_pos, false_pos)
auc

0.602633215840763

Get the false positive rate of models:

In [136]:
mt_test = mt_test.filter_rows(mt_test.cadd > 10)

In [137]:
#mt_test.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-test-cadd-10.mt')

In [173]:
mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-test-cadd-10.mt')

In [45]:
#mt_for_false_pos_10 = mt_for_skat.filter_rows(mt_for_skat.cadd > 10)

In [47]:
#mt_for_false_pos_10.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-false-pos-10.mt')

2020-11-03 17:08:56 Hail: INFO: wrote matrix table with 585211 rows and 78 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/mt_for_false_pos_10


In [174]:
mt_for_false_pos_10 = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-false-pos-10.mt')

In [ ]:
genes_background = mt_test.nearest_genes_20kb.collect()

In [ ]:
genes_background_1 =  set([val for sublist in genes_background for val in sublist])

In [24]:
len(genes_background_1)

55253

In [25]:
len(allgenes)

22845

In [26]:
genes_background = [x for x in genes_background_1 if x in allgenes]

len(genes_background)

18122

In [28]:
type(genes_background)

list

In [30]:
genes_background = np.array(genes_background)

In [33]:
#np.save('genes_background', genes_background)

In [30]:
genes_background = np.load('numpy/genes_background.npy')

In [180]:
false_pos_test = np.zeros((len(range(0,100)), len(np.linspace(-25,25,1000))))
true_pos_test = np.zeros((len(range(0,100)), len(np.linspace(-25,25,1000))))

In [181]:
categories = mt_for_skat.category.collect()
categories_test = mt_test.category.collect()

In [ ]:
for gene in range(0,100):

    randoms = np.random.randint(len(genes_background), size=6)
    geneset = [j for i, j in enumerate(genes_background) if i in randoms]

    print(geneset)
    
    test_asignment = np.zeros((6, 144))
    
    for rows, n in enumerate(geneset):

        mt_skat_log = mt_for_false_pos_10.filter_rows(mt_for_false_pos_10.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
        mt_test_skat_log = mt_test.filter_rows(mt_test.nearest_genes_20kb.contains(n)) # to ma rodziny
    
        mt_skat_log = mt_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
        non_refs = mt_skat_log.non_refs.collect()
        
        mt_test_skat_log = mt_test_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log.GT.is_non_ref())) #count variants per sample, prepare also test matrix
        non_refs_test = mt_test_skat_log.non_refs.collect()
        
        variants_controls = np.mean(np.array(non_refs)[np.invert(categories)])
        print(variants_controls)
        
        test_asignment[rows] = (variants_controls - non_refs_test)
 
    test_asignment = np.sum(test_asignment, axis = 0)  
    print(test_asignment.shape)
    
    a = []
    b = []
    
    for idx, i in enumerate(np.linspace(-25,25,1000)): 
        
        a.append(np.sum((test_asignment > i)[np.invert(categories_test)])/53)
        b.append(np.sum((test_asignment > i)[categories_test])/91)
    
    
    false_pos_test[gene] = a
    true_pos_test[gene] = b
  
    print('I have completed iteration number: ' + str(gene))


['ISY1', 'SIRPG', 'DCBLD2', 'IFI16', 'CNBD2', 'TTC34']
1.0256410256410255
1.2307692307692308
7.717948717948718
0.5641025641025641
2.3076923076923075
0.8974358974358975
(144,)
I have completed iteration number: 0
['THAP7', 'MYOM2', 'RGS21', 'OR5A1', 'TEX264', 'REP15']
5.102564102564102
5.282051282051282
1.3846153846153846
3.948717948717949
2.1025641025641026
1.5897435897435896
(144,)
I have completed iteration number: 1
['MRPL55', 'LEAP2', 'HIST1H2BM', 'HPD', 'LYN', 'FAM53C']
0.3333333333333333
4.0
0.8974358974358975
1.0512820512820513
4.051282051282051
7.9743589743589745
(144,)
I have completed iteration number: 2
['RGS11', 'SPTLC2', 'TMTC1', 'UBE2V1', 'NRG2', 'CPAMD8']
7.461538461538462
8.666666666666666
10.717948717948717
1.6923076923076923
7.717948717948718
1.4615384615384615
(144,)
I have completed iteration number: 3
['NCEH1', 'ASTN2', 'PZP', 'MRPS9', 'PAGE5', 'NCOA3']
5.564102564102564
62.282051282051285
4.9743589743589745
9.051282051282051
0.0
3.8974358974358974
(144,)
I have co

In [77]:
#np.save('false_pos_test', false_pos_test)
#np.save('true_pos_test', true_pos_test)

In [78]:
#np.save('false_pos_test_2', false_pos_test)
#np.save('true_pos_test_2', true_pos_test)

In [37]:
false_pos_test = np.load('numpy/false_pos_test.npy')
true_pos_test = np.load('numpy/true_pos_test.npy')

In [38]:
false_pos_test_2 = np.load('numpy/false_pos_test_2.npy')
true_pos_test_2 = np.load('numpy/true_pos_test_2.npy')

In [40]:
false_pos_test = np.concatenate((false_pos_test, false_pos_test_2))
true_pos_test = np.concatenate((true_pos_test, true_pos_test_2))

## AUC

In [41]:
aucs = []
for i in range(0,100):
    aucs.append(np.trapz(true_pos_test[i,], false_pos_test[i,]))

In [42]:
aucs = np.array(aucs)

In [44]:
np.percentile(-aucs, 95)

0.5940182459050383

In [45]:
test1 = np.array(([0,1,2,4,5], [0,2,2,3,5]))
test2 = np.array(([1,1,1,1,1], [2,2,2,2,2]))

In [46]:
y = np.linspace(0,1,10)
x = np.linspace(0,1,10)

In [89]:
p4 = figure(plot_width=800, plot_height=800)
p4.line(x, y, line_width=4, line_color='lightgrey')


for i in range(0,100):
    p4.line(false_pos_test[i,], true_pos_test[i,], line_width=1, alpha=0.25)
    
p4.line(false_pos, true_pos, line_width=4, line_color='orange')

p4.xaxis.axis_label = 'false positives'
p4.yaxis.axis_label = 'true positives'

p4.yaxis.axis_label_text_font_size = "25px"
p4.xaxis.axis_label_text_font_size = "25px"

# show the results
show(p4)

NameError: name 'x' is not defined

## investigate variants that went into the model

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/GTS-gnomad-sex.mt')

In [ ]:
top_genes = ['HDC', 'CHADL', 'MAOA', 'NAA11']

In [95]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(top_genes).contains(x), mt.nearest_genes_20kb))

In [96]:
mt.count()

(1230, 370)

In [97]:
mt = mt.filter_rows(mt.cadd > 10)

In [98]:
mt.count()

(53, 370)

In [100]:
#mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/top-variants.mt')

2020-11-09 17:54:13 Hail: INFO: wrote matrix table with 53 rows and 370 columns in 6622 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/top-variants.mt


In [188]:
top = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/top-variants.mt')

In [177]:
top_genes

['HDC', 'CHADL', 'MAOA', 'NAA11']

In [68]:
top = top.annotate_rows(
                      all_gnomads_non_ref = hl.agg.filter((top.phenotypes.phenotype == 'gnomad'), hl.agg.count_where(top.GT.is_non_ref()))/185,
                      all_gnomads_hom_var = hl.agg.filter((top.phenotypes.phenotype == 'gnomad'), hl.agg.count_where(top.GT.is_hom_var()))/185,
                      controls_non_ref = hl.agg.filter((top.phenotypes.disease == 'NO'), hl.agg.count_where(top.GT.is_non_ref()))/53,
                      controls_hom_var = hl.agg.filter((top.phenotypes.disease == 'NO'), hl.agg.count_where(top.GT.is_hom_var()))/53,
                      gts_all_non_ref = hl.agg.filter((top.phenotypes.disease == 'YES'), hl.agg.count_where(top.GT.is_non_ref()))/130,
                      gts_all_hom_var = hl.agg.filter((top.phenotypes.disease == 'YES'), hl.agg.count_where(top.GT.is_hom_var()))/130)

# controls = 53 gts = 130 gnomad 185

In [73]:
top = top.rows()

In [75]:
top = top.to_pandas()

In [78]:
top.to_csv('/net/archive/groups/plggneuromol/GTS-analysis/data/top-variants.csv')

## Run SKAT on all genes

In [11]:
#reimport the genes table again

allgenes = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/gts_gene_lists/human-genes-with-GO-and-symbols') 
allgenes = allgenes.select('UniProtKB Gene Name symbol')

allgenes = allgenes['UniProtKB Gene Name symbol'].collect()

2020-11-13 10:02:18 Hail: INFO: Reading table with no type imputation
  Loading column 'Gene stable ID' as type 'str' (type not specified)
  Loading column 'UniProtKB Gene Name symbol' as type 'str' (type not specified)



In [45]:
skat_table_all, qq_plot_all, top_genes_all, false_pos_all, true_pos_all, p_all, auc_all = full_model(allgenes)

2020-11-12 12:50:48 Hail: INFO: hwe_normalized_pca: running PCA using 10045 variants.
2020-11-12 12:50:59 Hail: INFO: pca: running PCA with 10 components...
2020-11-12 12:54:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 12:57:01 Hail: INFO: Ordering unsorted dataset with network shuffle


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""AACS""",246,7.51e+02,9.81e-04,0
"""ABCA7""",233,3.73e+03,3.36e-08,0
"""ADAMTSL4""",87,7.94e+02,9.69e-04,0
"""ADCY5""",483,3.21e+03,3.11e-05,0
"""ADGRL4""",1088,5.47e+03,8.34e-04,0
"""AFTPH""",152,1.51e+03,9.70e-04,0
"""AGK""",153,5.90e+02,8.17e-04,0
"""AIFM1""",68,2.79e+03,1.47e-05,0
"""AMIGO3""",78,6.84e+02,5.71e-04,0


2020-11-12 12:59:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 12:59:53 Hail: INFO: Ordering unsorted dataset with network shuffle


2020-11-12 13:03:10 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 13:05:54 Hail: INFO: Ordering unsorted dataset with network shuffle


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""AACS""",246,7.51e+02,9.81e-04,0
"""ABCA7""",233,3.73e+03,3.36e-08,0
"""ADAMTSL4""",87,7.94e+02,9.69e-04,0
"""ADCY5""",483,3.21e+03,3.11e-05,0
"""ADGRL4""",1088,5.47e+03,8.34e-04,0
"""AFTPH""",152,1.51e+03,9.70e-04,0
"""AGK""",153,5.90e+02,8.17e-04,0
"""AIFM1""",68,2.79e+03,1.47e-05,0
"""AMIGO3""",78,6.84e+02,5.71e-04,0


2020-11-12 13:08:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 13:08:49 Hail: INFO: Ordering unsorted dataset with network shuffle


2020-11-12 13:12:02 Hail: INFO: Ordering unsorted dataset with network shuffle


In [47]:
auc_all

0.45998341281360156

## run model on other gene lists

- brain enriched
- other gene lists

In [48]:
genes_neuro = list(set([line.rstrip('\n') for line in open('/net/archive/groups/plggneuromol/GTS-analysis/analysis/gts_gene_lists/brain_enriched.txt')]))

In [49]:
len(genes_neuro)

488

In [50]:
skat_table_neuro, qq_plot_neuro, top_genes_neuro, false_pos_neuro, true_pos_neuro, p_neuro, auc_neuro = full_model(genes_neuro)

2020-11-12 14:18:07 Hail: INFO: hwe_normalized_pca: running PCA using 10045 variants.
2020-11-12 14:18:18 Hail: INFO: pca: running PCA with 10 components...
2020-11-12 14:19:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 14:20:48 Hail: INFO: Ordering unsorted dataset with network shuffle


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHRM5""",282,9.55e+02,1.53e-03,0
"""GRM3""",399,2.53e+03,9.65e-04,0
"""KCNK4""",81,1.32e+03,1.01e-04,0
"""MTURN""",198,1.14e+03,1.19e-03,0
"""NEUROD1""",75,6.82e+02,1.46e-03,0
"""PDZD4""",95,1.20e+03,1.55e-04,0
"""S100B""",103,6.54e+02,7.30e-05,0
"""SCG3""",152,9.85e+02,1.05e-03,0


2020-11-12 14:21:45 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 14:22:00 Hail: INFO: Coerced sorted dataset
2020-11-12 14:22:00 Hail: INFO: Coerced dataset with out-of-order partitions.


2020-11-12 14:22:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 14:23:39 Hail: INFO: Ordering unsorted dataset with network shuffle


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHRM5""",282,9.55e+02,1.53e-03,0
"""GRM3""",399,2.53e+03,9.65e-04,0
"""KCNK4""",81,1.32e+03,1.01e-04,0
"""MTURN""",198,1.14e+03,1.19e-03,0
"""NEUROD1""",75,6.82e+02,1.46e-03,0
"""PDZD4""",95,1.20e+03,1.55e-04,0
"""S100B""",103,6.54e+02,7.30e-05,0
"""SCG3""",152,9.85e+02,1.05e-03,0


2020-11-12 14:24:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-12 14:24:42 Hail: INFO: Coerced sorted dataset
2020-11-12 14:24:42 Hail: INFO: Coerced dataset with out-of-order partitions.


2020-11-12 14:25:25 Hail: INFO: Ordering unsorted dataset with network shuffle


In [51]:
auc_neuro

0.46174580136844284

In [138]:
new_gene_lists = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/gts_gene_lists/custom_lists_gts.csv')

2020-11-17 12:45:43 Hail: INFO: Reading table with no type imputation
  Loading column 'neurotranmitters' as type 'str' (type not specified)
  Loading column 'glutamate' as type 'str' (type not specified)
  Loading column 'serotonine' as type 'str' (type not specified)
  Loading column 'dop_ach' as type 'str' (type not specified)
  Loading column 'GTS_genes' as type 'str' (type not specified)
  Loading column 'synaptic_genes' as type 'str' (type not specified)
  Loading column 'tryptofane' as type 'str' (type not specified)
  Loading column 'receptors' as type 'str' (type not specified)
  Loading column 'calcium' as type 'str' (type not specified)
  Loading column 'androgenic_receptor' as type 'str' (type not specified)
  Loading column 'addictions' as type 'str' (type not specified)



In [139]:
tra = new_gene_lists['neurotranmitters'].collect()
glut = new_gene_lists['glutamate'].collect()
ser = new_gene_lists['serotonine'].collect()
dop = new_gene_lists['dop_ach'].collect()
gts = new_gene_lists['GTS_genes'].collect()
syn = new_gene_lists['synaptic_genes'].collect()
tryp = new_gene_lists['tryptofane'].collect()
rec = new_gene_lists['receptors'].collect()
ca = new_gene_lists['calcium'].collect()
andr = new_gene_lists['androgenic_receptor'].collect()
add = new_gene_lists['addictions'].collect()

In [52]:
gene_lists = [tra, glut, ser, dop, gts, syn, tryp, rec, ca, andr, add]
list_aucs = []

In [ ]:
for gene in gene_lists:
    s, qq, top, fp, tp, p, auc = full_model(gene)
    list_aucs.append(auc)

In [17]:
#np.save('numpy/list_aucs', list_aucs)

In [55]:
list_aucs = np.load('numpy/list_aucs.npy')

In [62]:
list_aucs

array([0.49295045, 0.48227244, 0.50839726, 0.585113  , 0.52114866,
       0.50829359, 0.41986316, 0.51969728, 0.52840556])

In [180]:
s, qq, top, fp, tp, p, auc = full_model(dop)

2020-11-17 14:17:01 Hail: INFO: hwe_normalized_pca: running PCA using 10045 variants.
2020-11-17 14:17:05 Hail: INFO: pca: running PCA with 10 components...
2020-11-17 14:18:55 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-17 14:20:19 Hail: INFO: Ordering unsorted dataset with network shuffle


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHRM2""",422,1.83e+03,1.28e-03,0
"""CHRM5""",282,9.55e+02,1.53e-03,0
"""HDC""",118,5.46e+02,1.79e-03,0


2020-11-17 14:21:40 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-17 14:22:11 Hail: INFO: Coerced sorted dataset
2020-11-17 14:22:11 Hail: INFO: Coerced dataset with out-of-order partitions.


2020-11-17 14:23:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-17 14:24:55 Hail: INFO: Ordering unsorted dataset with network shuffle


id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHRM2""",422,1.83e+03,1.28e-03,0
"""CHRM5""",282,9.55e+02,1.53e-03,0
"""HDC""",118,5.46e+02,1.79e-03,0


2020-11-17 14:26:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-17 14:27:38 Hail: INFO: Coerced sorted dataset
2020-11-17 14:27:38 Hail: INFO: Coerced dataset with out-of-order partitions.


2020-11-17 14:28:47 Hail: INFO: Ordering unsorted dataset with network shuffle


In [63]:
list_aucs = np.concatenate((list_aucs, list_aucs_2))

In [64]:
list_aucs

array([0.49295045, 0.48227244, 0.50839726, 0.585113  , 0.52114866,
       0.50829359, 0.41986316, 0.51969728, 0.52840556, 0.50642753,
       0.41011818])

In [185]:
s.order_by(s.p_value).show(10)

2020-11-17 14:53:00 Hail: INFO: Ordering unsorted dataset with network shuffle


id,size,q_stat,p_value,fault,label
str,int32,float64,float64,int32,bool
"""CHRM2""",422,1.83e+03,1.28e-03,0,false
"""CHRM5""",282,9.55e+02,1.53e-03,0,false
"""HDC""",118,5.46e+02,1.79e-03,0,false
"""SCAMP2""",92,8.58e+02,2.08e-03,0,false
"""GNG2""",504,2.05e+03,2.39e-03,0,false
"""GAD1""",210,1.38e+03,3.53e-03,0,false
"""CHRM1""",56,3.05e+02,7.48e-03,0,false
"""HRH2""",124,1.82e+02,1.19e-02,0,false
"""MAOA""",95,6.96e+02,1.35e-02,0,false


In [64]:
#do this at the end for the mt_for_skat

def remove_related(mtx, mtx_subset):
    pc_rel = hl.pc_relate(mtx_subset.GT, 0.001, k=2, statistics='kin')
    pairs = pc_rel.filter(pc_rel['kin'] > 0.125)
    related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, keep=False)
    
    related_samples_to_remove = related_samples_to_remove.annotate(s = related_samples_to_remove.node.s)
    related_samples_to_remove = related_samples_to_remove.key_by('s')
    
    mtx = mtx.key_cols_by()
    mtx = mtx.filter_cols(hl.is_defined(related_samples_to_remove[mtx.s]), keep=False)
    mtx = mtx.key_cols_by(mtx.s)
    
    mtx_subset = mtx_subset.filter_cols(hl.is_defined(related_samples_to_remove[mtx_subset.s]), keep=False)
    
    return(mtx, mtx_subset)